In [2]:
from google.cloud import bigquery
client = bigquery.Client()

df = client.query("""
select *
from `greenlabs-data-farmmorning.content_analysis_us.gs_crawling_packer_info_processed` 
WHERE data_type = 'homepage'
             """).to_dataframe()

In [11]:

df.to_json(orient='records', force_ascii=False)

'[{"source_id":"gs:\\/\\/greenlabs-data-grainscanner\\/crawling\\/raw\\/auto_test.csv","uuid":"1$c2428b-9295-442d-94e8-cbad42a3c189","data_type":"homepage","job_type":"search","job_detail":"text-bison@001","raw_data":"AGRIGRAIN (COONAMBLE) PTY LTD","processed_data":"","embedding_array":[],"process_ts":1707289745290},{"source_id":"gs:\\/\\/greenlabs-data-grainscanner\\/crawling\\/raw\\/auto_test.csv","uuid":"6$c2428b-9295-442d-94e8-cbad42a3c189","data_type":"homepage","job_type":"search","job_detail":"text-bison@001","raw_data":"WATSON\'S GRAIN STORAGE AND PACKING PTY LTD","processed_data":"","embedding_array":[],"process_ts":1707289745290},{"source_id":"gs:\\/\\/greenlabs-data-grainscanner\\/crawling\\/raw\\/auto_test.csv","uuid":"5$c2428b-9295-442d-94e8-cbad42a3c189","data_type":"homepage","job_type":"search","job_detail":"text-bison@001","raw_data":"ROBINSON GRAIN TRADING CO PTY LTD","processed_data":"","embedding_array":[],"process_ts":1707289745290},{"source_id":"gs:\\/\\/greenlabs

In [19]:
import pandas as pd

new_df = pd.DataFrame([{
  "source_id": "gs://greenlabs-data-grainscanner/crawling/raw/auto_test.csv",
  "uuid": "2$c2428b-9295-442d-94e8-cbad42a3c189",
  "packer_name": "VITERRA LTD",
  "url": "https://www.viterra.com/",
  "is_required_addr": "true",
  "is_required_email": "true",
  "is_required_phone": "true",
  "is_required_sku": "true"
}])

In [26]:
import json
def make_col_list(row):
    
    result = []
    if row["is_required_addr"]:
        result.append('address')
    if row['is_required_email']:
        result.append('mail')
    if row['is_required_phone']:
        result.append('phone')
    if row['is_required_sku']:
        result.append('products')
    return json.dumps(result, ensure_ascii=False)
new_df['column_list'] = new_df.apply(make_col_list, axis=1)

In [27]:
new_df

,source_id,uuid,packer_name,url,is_required_addr,is_required_email,is_required_phone,is_required_sku,column_list
0,gs://greenlabs-data-grainscanner/crawling/raw/...,2$c2428b-9295-442d-94e8-cbad42a3c189,VITERRA LTD,https://www.viterra.com/,true,true,true,true,"[""address"", ""mail"", ""phone"", ""products""]"


In [30]:
df = pd.read_csv("/Users/yoonhae/Downloads/china_2023.csv")
df['기존 전화번호'] = df['기존 전화번호'].apply(lambda x: str(x) if pd.notnull(x) else x)
df.to_csv("/Users/yoonhae/Downloads/china_2023.csv", index=False, quoting=1)

In [4]:
import time
from google.cloud import pubsub_v1

def push_crawling_message_to_pubsub(topic_id, message):
    # GCP project_id와 topic_id 지정
    project_id = "greenlabs-data-farmmorning"

    # publisher client 생성
    # ordering_key 를 사용하는 경우 client 를 생성하면서 enable_message_ordering 옵션을 설정해야한다.
    # ordering_key 를 사용하지 않는다면 publisher = pubsub_v1.PublisherClient() 만 해도 무방하다.
    publisher = pubsub_v1.PublisherClient(
        publisher_options = pubsub_v1.types.PublisherOptions(
                #enable_message_ordering=True,
    ))

    # 게시하고자 하는 topic 의 경로를 구성한다.
    topic_path = publisher.topic_path(project_id, topic_id)

    # data schema 에 맞게 messsage 를 구성한다.
    message["timestamp"] = time.time()
    # 전송되는 데이터는 bytes 타입이다.
    # dict 형식의 데이터의 경우 string 으로 변환 후 byte 로 인코딩한다.
    data_str = json.dumps(message, ensure_ascii=False)
    data = data_str.encode("utf-8")

    # message 를 게시한다.
    # publish 의 기본 인자는 topic, data, ordering_key, retry, timeout 이며
    # 이후 키워드 인자로 입력되는 것은 attributes 로 등록된다.
    # 여기서는 attributes.product="recommendation_follows" 가 설정된다.
    future = publisher.publish(topic_path
                                , data
                                , target=message["target"]  # attributes
                            )
    # 메세지를 정상적으로 게시할때까지 block 된다.
    # 정상적으로 등록된 경우 messge ID 를 반환하고 실패한 경우 Exception 이 발생한다.
    # https://cloud.google.com/python/docs/reference/pubsub/latest/google.cloud.pubsub_v1.publisher.futures.Future
    print('** pub/sub push id :', future.result())
def make_col_list(row):
    
    result = []
    if row["is_required_addr"]:
        result.append('address')
    if row['is_required_email']:
        result.append('mail')
    if row['is_required_phone']:
        result.append('phone')
    if row['is_required_sku']:
        result.append('products')
    return result

In [17]:
import json 
query = """
with raw as (
  select source_id, uuid, packer_name, website_url
    , (coalesce(handled_sku1, '') = '' and coalesce(sku_raw, '') = '') as is_required_sku
    , (coalesce(country, '') = '' and coalesce(address_raw, '') = '') as is_required_addr
    , (coalesce(phone_number, '') = '' and coalesce(original_phone_number, '') = '') as is_required_phone 
    , coalesce(primary_email, '') = '' as is_required_email
  from `greenlabs-data-farmmorning.content_analysis_us.gs_crawling_packer_info_raw`
  where source_id = "gs://greenlabs-data-grainscanner/crawling/raw/china_2023.csv"
    and (
        (coalesce(handled_sku1, '') = '' and coalesce(sku_raw, '') = '') or
        (coalesce(country, '') = '' and coalesce(address_raw, '') = '') or
        (coalesce(phone_number, '') = '' and coalesce(original_phone_number, '') = '') or
        coalesce(primary_email, '') = ''
    )
)
, search_set as (
  select uuid, raw_data as packer_name, processed_data as website_url
  from `greenlabs-data-farmmorning.content_analysis_us.gs_crawling_packer_info_processed` 
  WHERE source_id = "gs://greenlabs-data-grainscanner/crawling/raw/china_2023.csv"
    and data_type = 'homepage' 
    and job_type = 'search'  
)
select raw.source_id
  , raw.uuid
  , raw.packer_name
  , coalesce(raw.website_url, search_set.website_url) as url
  , raw.is_required_addr
  , raw.is_required_email
  , raw.is_required_phone
  , raw.is_required_sku
from raw
left join search_set
on raw.uuid = search_set.uuid
where coalesce(raw.website_url, search_set.website_url) > ''
order by uuid
limit 1
"""
from google.cloud import bigquery

bigquery_client = bigquery.Client()
df = bigquery_client.query(query).to_dataframe()

df['column_list'] = df.apply(make_col_list, axis=1)
topic = "crawler_jobs_topic"
for index, row in df.iterrows():
    message = {
        "target": "packer_homepage",
        "target_column_list": json.dumps(row['column_list'], ensure_ascii=False),
        "source_id": row['source_id'],
        'uuid': row["uuid"],
        "homepage_url": row["url"],
        "write_bigquery_path": 'greenlabs-data-farmmorning.content_analysis_us.gs_crawling_packer_info_processed'
    }
    push_crawling_message_to_pubsub(topic, message)

** pub/sub push id : 9214710409184166


In [8]:
data = '[{"homepage":"https:\/\/aaawholesale.com\/","extract_url":["https:\/\/aaawholesale.com\/","https:\/\/aaawholesale.com\/contact-2\/","https:\/\/aaawholesale.com\/about-us\/"],"company_name":[],"mail":["[email protected]"],"phone":["201-351-5000"],"address":["35 Oxford Drive Moonachie, NJ 07074","35 Oxford Dr. Moonachie, NJ  07074 USA"],"products":[],"error":[]}]'

j_data = json.loads(data)

In [16]:
import pandas as pd
sub_df = pd.DataFrame(j_data)
print(json.dumps(sub_df.to_dict(orient='records'), ensure_ascii=False))
print(json.dumps(j_data))


[{"homepage": "https://aaawholesale.com/", "extract_url": ["https://aaawholesale.com/", "https://aaawholesale.com/contact-2/", "https://aaawholesale.com/about-us/"], "company_name": [], "mail": ["[email protected]"], "phone": ["201-351-5000"], "address": ["35 Oxford Drive Moonachie, NJ 07074", "35 Oxford Dr. Moonachie, NJ  07074 USA"], "products": [], "error": []}]
[{"homepage": "https://aaawholesale.com/", "extract_url": ["https://aaawholesale.com/", "https://aaawholesale.com/contact-2/", "https://aaawholesale.com/about-us/"], "company_name": [], "mail": ["[email protected]"], "phone": ["201-351-5000"], "address": ["35 Oxford Drive Moonachie, NJ 07074", "35 Oxford Dr. Moonachie, NJ  07074 USA"], "products": [], "error": []}]


In [7]:
job_id = 'airflow_1709190130022928_403f1349cd8e2126ade7d5c09d649e25'
path = '/Users/yoonhae/data/.gcp-secure/greenlabs-data-farmmorning.json'
from google.cloud import bigquery
bigquery_client = bigquery.Client.from_service_account_json(path)

# job_id를 사용하여 BigQuery 작업 가져오기
job = bigquery_client.get_job(job_id)  # API 요청
df = job.result().to_dataframe() 


NotFound: 404 GET https://bigquery.googleapis.com/bigquery/v2/projects/greenlabs-data-farmmorning/jobs/airflow_1709190130022928_403f1349cd8e2126ade7d5c09d649e25?projection=full&prettyPrint=false: Not found: Job greenlabs-data-farmmorning:airflow_1709190130022928_403f1349cd8e2126ade7d5c09d649e25